## Web scraping from IMDB pages

In [1]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import numpy as np
import json

In [2]:
url = "https://www.imdb.com/list/ls068010962/"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html5lib') 
# print(soup.prettify)

In [3]:
import re
p=soup.find(type="application/ld+json")
text=str(p);tex=text.split("\n");

#### Finding url of pages corresponding to each star

In [ ]:
text=[]
for t in tex:
    if re.search(r"\"url\": *", t):
        t=t.split("\"url\":",1)[1];
        t=t.strip();t=t[1:-1]
        text.append("https://www.imdb.com"+t)

#### Obtaining all required parameters from celeb's IMDB page through scraping

In [4]:
df=[];j=0
for te in text:
    r = requests.get(te)
    soup = BeautifulSoup(r.content, 'html5lib') 
    p=soup.find(type="application/ld+json")
    t1=str(p);t1=t1.split("\n");
    data={}
    
    for t in t1:
        if re.search(r"\"name\": *", t):
            t=t.split("\"name\":",1)[1];# Obtaining name [content given after "name":]
            t=t.strip();t=re.findall(r'"([^"]*)"',t)[0]
            data['Name']=t      
        if re.search(r"\"image\": *", t):
            t=t.split("\"image\":",1)[1];t=re.findall(r'"([^"]*)"',t)[0]
            data['Image']=t     
        if re.search(r"\"description\": *", t):
            t=t.split("\"description\":",1)[1];t=re.findall(r'"([^"]*)"',t)[0]
            data['Description']=t  
        if re.search(r"\"birthDate\": *", t):
            t=t.split("\"birthDate\":",1)[1];t=re.findall(r'"([^"]*)"',t)[0]
            data['Birthdate']=t
    df.append(data)
df=pd.DataFrame(df)#converting to pandas DataFrame
df.head()

,Birthdate,Description,Image,Name
0,1965-03-14,Aamir is no doubt one of the most dedicated ac...,https://m.media-amazon.com/images/M/MV5BMjAwMj...,Aamir Khan
1,1967-09-09,"Akshay Kumar, born September 9, 1967, is an ac...",https://m.media-amazon.com/images/M/MV5BODI4ND...,Akshay Kumar
2,1969-04-02,"Ajay Devgn is an Indian film actor, director, ...",https://m.media-amazon.com/images/M/MV5BMTY0MD...,Ajay Devgn
3,1940-11-12,"Amjad Khan was born on November 12, 1940 in Hy...",https://m.media-amazon.com/images/M/MV5BOWUzNj...,Amjad Khan
4,1942-10-11,Son of well known poet Harivansh Rai Bachchan ...,https://m.media-amazon.com/images/M/MV5BNTk1OT...,Amitabh Bachchan


In [5]:
#Saving the database 
df.to_csv("Movie_celeb_IMDB.csv")